In [1]:
import os
import datetime
import lxml.etree as et
import csv
pmcdir = 'pmc_articles'
from plos_corpus import (corpusdir, get_article_pubdate, check_if_uncorrected_proof, listdir_nohidden,
                         get_article_xml, file_to_doi, doi_to_file, get_all_solr_dois, download_check_and_move)

from samples.corpus_analysis import (get_plos_article_type, get_article_dtd, get_random_list_of_dois, 
                                     get_related_retraction_article, check_article_type, get_plos_journal,
                                     get_article_title, parse_article_date, get_corpus_metadata,
                                     get_article_abstract, corpus_metadata_to_csv, get_article_dates,
                                     read_corpus_metadata_from_csv)


In [ ]:
article_files = [doi_to_file(doi) for doi in get_random_list_of_dois(count=1000)]


In [ ]:
corpus_metadata = get_corpus_metadata(article_files)

In [ ]:
corpus_metadata_to_csv(corpus_metadata=corpus_metadata)

In [ ]:
one_list = [article for article in listdir_nohidden(corpusdir) if 'pone' in article]

def assemble_diabetes_corpus(article_list):
    """
    Find all PLOS ONE articles that say something about diabetes or technology in the abstract.
    """
    diabetes_article_list = [article for article in article_list if 'diabet' in get_article_abstract(article).lower()]
    return diabetes_article_list

In [ ]:
diabetes_article_list = assemble_diabetes_corpus(one_list)

In [ ]:
diabetes_metadata = get_corpus_metadata(article_list=diabetes_article_list)

In [ ]:
corpus_metadata_to_csv(diabetes_meta_truncated)

In [ ]:
diabetes_meta_truncated = []
for row in diabetes_metadata:
    diabetes_meta_truncated.append(row[:-1])

In [ ]:
len(diabetes_meta_truncated)

In [ ]:
corpus_metadata_remade = read_corpus_metadata_from_csv()

In [ ]:
csv_doi_list = [row[0] for row in corpus_metadata_remade]

In [ ]:
len(csv_doi_list)

In [ ]:
full_doi_list = get_all_solr_dois()

In [ ]:
dois_needed_list = list(set(full_doi_list) - set(csv_doi_list))

In [ ]:
len(dois_needed_list)

In [ ]:
print(len(corpus_metadata_remade))
for doi in dois_needed_list[0:10]:
    article_metadata = get_article_metadata(doi)
    corpus_metadata_remade.append(article_metadata)
print(len(corpus_metadata_remade))

In [ ]:
doi_list = dois_needed_list[0:100]

In [ ]:
corpus_metadata = update_corpus_metadata_csv(comparison_dois=doi_list)

In [ ]:
a = [(1,2), (3,4)]
b = [(5,6), (7,8)]

In [ ]:
a.append(b)

In [ ]:
a

In [ ]:
'allofplos_xml/journal.ppat.1006658.xml'.split('/')[1]

In [ ]:
os.path.join('New articles', 'allofplos_xml/journal.ppat.1006658.xml'.split('/')[1].rstrip('.xml')+'.xml')

In [2]:
sample_article_list = get_random_list_of_dois(count=50000)

In [3]:
corpus_metadata = get_corpus_metadata(article_list=sample_article_list)

  2% (1046 of 50000) |                    | Elapsed Time: 0:01:08 ETA:  0:53:35

10.1371/journal.pntd.0001753 dates not in correct order: {'collection': datetime.datetime(2012, 7, 1, 0, 0), 'epub': datetime.datetime(2012, 7, 24, 0, 0), 'received': datetime.datetime(2012, 12, 17, 0, 0), 'accepted': datetime.datetime(2012, 6, 12, 0, 0)}


  7% (3776 of 50000) |#                   | Elapsed Time: 0:03:31 ETA:  0:42:16

10.1371/journal.pntd.0001618 dates not in correct order: {'collection': datetime.datetime(2012, 5, 1, 0, 0), 'epub': datetime.datetime(2012, 5, 1, 0, 0), 'received': datetime.datetime(2012, 12, 30, 0, 0), 'accepted': datetime.datetime(2012, 3, 5, 0, 0)}


 24% (12434 of 50000) |####               | Elapsed Time: 0:10:27 ETA:  0:28:57

...

 80% (40289 of 50000) |###############    | Elapsed Time: 0:35:17 ETA:  0:09:06

10.1371/journal.pntd.0003959 dates not in correct order: {'epub': datetime.datetime(2015, 7, 28, 0, 0), 'collection': datetime.datetime(2015, 7, 1, 0, 0), 'received': datetime.datetime(2015, 10, 5, 0, 0), 'accepted': datetime.datetime(2015, 7, 7, 0, 0)}


 97% (48760 of 50000) |################## | Elapsed Time: 0:42:55 ETA:  0:01:14

10.1371/journal.pcbi.0020027 dates not in correct order: {'ppub': datetime.datetime(2006, 4, 1, 0, 0), 'epub': datetime.datetime(2006, 4, 14, 0, 0), 'received': datetime.datetime(2005, 12, 22, 0, 0), 'accepted': datetime.datetime(2005, 2, 20, 0, 0)}


100% (50000 of 50000) |###################| Elapsed Time: 0:44:05 Time: 0:44:05


In [4]:
corpus_metadata_to_csv(corpus_metadata=corpus_metadata)

In [ ]:
#to execute: update corpus_metadata with a list of DOIs. check that functions can handle an overlapping list, and make
# sure that appending is working correctly. there were some errors reading in a csv that had been appended/extended.

In [6]:
file_to_doi('10.1371/journal.pone.0080675')

'10.1371/journal.pone'